In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import transformers
import os, json, glob, requests, torch, cv2
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing import image

In [51]:
#set default path
TRAIN_JSON_PATH = '/content/drive/MyDrive/감정따뜻쟁이/label/train/'
VALID_JSON_PATH = '/content/drive/MyDrive/감정따뜻쟁이/label/val/'
TEST_JSON_PATH = '/content/drive/MyDrive/감정따뜻쟁이/TEST_DATA_SET/label/'

TRAIN_IMG_PATH = '/content/drive/MyDrive/감정따뜻쟁이/img/train/'
VALID_IMG_PATH = '/content/drive/MyDrive/감정따뜻쟁이/img/val/'
TEST_IMG_PATH = '/content/drive/MyDrive/감정따뜻쟁이/TEST_DATA_SET/TEST_DATA_SET/'

TRAIN_SEGMENT_PATH = 'data/segmentation/train/'
VALID_SEGMENT_PATH = 'data/segmentation/val/'

IMG_ROWS = 256
IMG_COLS = 256

In [3]:
def get_json_list_1d(json_folder_path): #returns a 1 dim array
    temp_loader = []
    json_train = []
    emotion_list_json = os.listdir(json_folder_path)

    for emotion in emotion_list_json:
        with open (json_folder_path + emotion, "r", encoding='euc-kr') as f:
            temp_loader.append(json.load(f))

    for i in range(len(temp_loader)):
        json_train += temp_loader[i]

    return json_train

In [4]:
json_train = get_json_list_1d(TRAIN_JSON_PATH)
json_val = get_json_list_1d(VALID_JSON_PATH)

In [52]:
json_test = get_json_list_1d(TEST_JSON_PATH)

In [5]:
def load_image_and_label_1D(json_list, img_path):
    img_file = []
    label_list = []

    eng_to_num = { '분노'   : '0',
                   '기쁨'   : '1',
                   '당황'   : '2',
                   '슬픔'   : '3'
                 }
    kr_to_en = { '분노'    : 'anger',
                 '기쁨'   : 'happy',
                 '당황'   : 'panic',
                 '슬픔'   : 'sadness'
                 }


    for entry in range(len(json_list)):
        try:
            label = json_list[entry]['faceExp_uploader']
            img_dir = img_path + kr_to_en[label] + '/' + json_list[entry]['filename']
            # img = np.array(image.load_img(img_dir, target_size=(IMG_ROWS, IMG_COLS)))/255
            img = image.load_img(img_dir, target_size=(IMG_ROWS, IMG_COLS))
            img_file.append(img)
            label_list.append(int(eng_to_num[label]))
        except Exception:
            print("Emotion " + label + " " + json_list[entry]['filename'] + " Not available")
            pass

    return img_file, label_list

In [6]:
train_img, train_label = load_image_and_label_1D(json_train, TRAIN_IMG_PATH)

Emotion 기쁨 7zwecfe6e57f975ee8b238b355e3e87923c4557383daae96720c1b8c3f669g9m2.jpg Not available
Emotion 기쁨 oxmt62fbb28fafcdaa8d64e75a332e1d506d6a079cb45fc9e00c576c40fbeo0wy.jpg Not available
Emotion 기쁨 s1mu862ea97f26b2720eeecb51156f6e32e8523fd62f3f04ef2f727639f52d7g3.jpg Not available
Emotion 당황 twdva8d723865fa1005873a6967797c75f3dd83197f13f7daa3e3926947071al6.jpg Not available
Emotion 당황 5yid6d7ae32bf14b1343da46cfdb3fdffc8e1c64baf4fb1b175690f648a870v69.jpg Not available
Emotion 당황 bpv531e5cebdb1c00085158bb0677c5596ccabf1c8a5cb0007991571bf306wcd4.jpg Not available
Emotion 당황 s1q397a38900c1e1f950496109900e7c01f0630459790d8590010655c4090vlh0.jpg Not available
Emotion 당황 7m2k46c05803855399ea1ee1911875ee59e05640c93135c0bc91b5660456cdo8k.jpg Not available
Emotion 당황 n55yc741f09701506c7eb5ba30533db35c905ab5bf0597b5652ece03c5950ozv1.jpg Not available
Emotion 당황 bid7b3c93409e1d83514689337f6d3d63a338339d71c9ce7fded153d3576fk2ml.jpg Not available
Emotion 당황 if0lfc4b517c6597e3656dbea350e406e1b5ae4

In [53]:
test_img, test_label = load_image_and_label_1D(json_test, TEST_IMG_PATH)

Emotion 기쁨 u7g6433c6735ff35c2035b8f7437798514f0133b7ec2eb3c2ce2e935d0d46irg7.jpg Not available
Emotion 기쁨 ywv8a77ca199c4fb2434b6e43bbeb2b49057b4169ab645613f41794b93931008z.jpg Not available
Emotion 기쁨 u2mabc4b11aadab1fca6c2e816dd137e9ac511b461f5d649d40d9c36a241bpwv8.jpg Not available
Emotion 기쁨 yq2ne1ac4ec0ae73f4c46af88107352034ee636e3141813f1190f7ca21a9fyv89.jpg Not available
Emotion 기쁨 srnd22fa8c292b8bf0583101cec9c22189109fa81ad2ef941092fb29d29b8pt8e.jpg Not available
Emotion 기쁨 svulef3dbdc179db3b0d4190acddbac82cadd6a25ecb33f85cc769d49ab6ct1cp.jpg Not available
Emotion 기쁨 xxk14368611419b461c71803876671ca263e013106e600476e62c0c380ecewg4p.jpg Not available
Emotion 기쁨 vj2rbe218f64439e001e6d6e96df105de1666b0664e0e788b9e57e081800ez422.jpg Not available
Emotion 기쁨 x7x172ab122b8de88e11890797bbd22b4b281610f1bb278e822b2ba80dbc7kb0g.jpg Not available
Emotion 기쁨 wpce7bfafd07ca2f0bc82bb0bf2c3b0add82d7f13fb8667c2eb2bd2fcf2ebq8a9.jpg Not available
Emotion 기쁨 z3uzca6deee2155aa4a2bc316c9cff249cf3cda

In [7]:
val_img, val_label = load_image_and_label_1D(json_val, VALID_IMG_PATH)

Emotion 슬픔 b14secfe2b1749c94c7cc563020df10e40372c7b9c8dbe793779c6c2cb272cznr.jpg Not available


In [16]:
train_images = np.array(train_img)
train_labels = np.array(train_label)
val_images = np.array(val_img)
val_labels = np.array(val_label)

In [17]:
train_images = train_images / 255.0
val_images = val_images / 255.0

In [54]:
test_images = np.array(test_img)
test_labels = np.array(test_label)

test_images = test_images / 255.0

In [31]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


label_encoder = LabelEncoder()

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = train_labels.reshape(len(train_labels), 1)
train_labels = onehot_encoder.fit_transform(integer_encoded)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [55]:
integer_encoded_test = test_labels.reshape(len(test_labels), 1)
test_labels = onehot_encoder.transform(integer_encoded_test)

In [32]:
integer_encoded_val = val_labels.reshape(len(val_labels), 1)
val_labels = onehot_encoder.transform(integer_encoded_val)

In [35]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout,Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

#모델 불러오기
base_model = Xception(weights='imagenet', include_top=False, input_shape=(train_images.shape[1], train_images.shape[2], train_images.shape[3]))


# 새로운 분류층 추가
x = base_model.output
x = Conv2D(128, (3, 3), activation = 'relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Conv2D(64, (3, 3), activation = 'relu')(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 구조 확인
model.summary()

# 모델 학습
history = model.fit(train_images, train_labels, epochs=30, validation_data=(val_images, val_labels), batch_size=32)

# 모델 평가 (정확도 출력)
loss, accuracy = model.evaluate(val_images, val_labels)
print("정확도:", accuracy)

# history 객체를 통해 평균 정확도 계산
average_accuracy = np.mean(history.history['val_accuracy'])
print("평균 검증 정확도:", average_accuracy)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 127, 127, 32)         864       ['input_13[0][0]']            
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 127, 127, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 127, 127, 32)         0         ['block1_conv1_bn[0][0]'

In [56]:
# 테스트 이미지 데이터셋으로 모델 예측값 확인
predictions = model.predict(test_images)

19/19 [==============================] - 3s 125ms/step


In [57]:
print(predictions)

[[5.4137533e-10 1.0000000e+00 2.6828967e-14 2.0674275e-11]
 [1.4040004e-10 1.0000000e+00 1.2803816e-15 1.7877179e-12]
 [3.3145068e-19 1.0000000e+00 5.1088968e-26 3.2269832e-21]
 ...
 [9.9999988e-01 6.9049078e-10 4.6746376e-09 9.9072977e-08]
 [1.2182969e-01 4.6265898e-03 8.6176807e-01 1.1775637e-02]
 [9.9997890e-01 3.0586743e-06 6.2750350e-06 1.1720535e-05]]


In [58]:
# 예측 클래스 선택
predicted_classes = np.argmax(predictions, axis=1)

# 최종 예측값 출력
print(predicted_classes)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 0 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 0 2 2 2 2
 2 2 2 2 2 2 2 2 2 0 2 2 0 2 2 2 2 3 1 3 0 3 1 3 3 1 3 3 1 3 3 2 2 0 3 0 3
 3 3 0 0 3 3 3 3 0 3 3 3 0 3 3 3 0 0 0 0 2 0 0 2 2 0 0 0 0 0 2 0 0 0 0 0 0
 1 0 0 0 3 1 0 0 0 1 0 0 1 0 0 0 2 2 0 0 2 3 0 2 3 3 0 2 0 0 2 0 0 0 0 0 0
 2 3 0 0 0 2 1 0 3 0 2 0 0 1 3 2 2 2 2 2 0 0 2 0 1 0 0 2 3 0 2 0 2 3 2 0 2
 0 0 0 3 0 0 2 2 0 0 2 0 0 0 0 2 0 0 0 0 3 0 2 3 0 0 1 0 0 0 0 2 0 0 2 0 0
 0 2 2 2 0 1 0 2 2 0 0 2 3 0 0 0 0 0 0 3 0 0 1 0 0 0 0 0 0 0 0 2 0 0 0 0 0
 0 0 2 1 1 0 0 0 1 0 0 0 

In [59]:
# 예측 클래스와 실제 라벨 비교하여 정확도 계산
accuracy = np.mean(predicted_classes == np.argmax(test_labels, axis=1))

# 정확도 출력
print("정확도:", accuracy)

정확도: 0.7820945945945946
